In [1]:
!pip install tensorflow_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import os
import pathlib
import joblib

import numpy as np
import matplotlib.pyplot as plt

import cv2
import PIL

import tensorflow as tf
import tensorflow_hub as hub

In [14]:
train_dir = pathlib.Path("/content/drive/MyDrive/Data sets/Plant_leave_disease_dataset_with_augmentation/Pepper/train")
test_dir = pathlib.Path("/content/drive/MyDrive/Data sets/Plant_leave_disease_dataset_with_augmentation/Pepper/test")
val_dir = pathlib.Path("/content/drive/MyDrive/Data sets/Plant_leave_disease_dataset_with_augmentation/Pepper/val")

In [15]:
class_images = {
    "bacterial_spot": list(train_dir.glob("bacterial_spot/*")), 
    "healthy": list(train_dir.glob("healthy/*")),
}

class_labels = {
    "bacterial_spot": 1,
    "healthy": 0,
}

X, y = [], []

for image_name, images in class_images.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img, (224, 224))
        X.append(resized_img)
        y.append(class_labels[image_name])
        
X = np.array(X)
y = np.array(y)

In [16]:
feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

pretrained_model_without_top_layer = hub.KerasLayer(
    feature_extractor_model, input_shape=(224, 224, 3), trainable=False)

In [17]:
model = tf.keras.Sequential([
  pretrained_model_without_top_layer,
  tf.keras.layers.Dense(2)
])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_2 (KerasLayer)  (None, 1280)              2257984   
                                                                 
 dense_2 (Dense)             (None, 2)                 2562      
                                                                 
Total params: 2,260,546
Trainable params: 2,562
Non-trainable params: 2,257,984
_________________________________________________________________


In [18]:
model.compile(
  optimizer="adam",
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'], run_eagerly=True)

model.fit(X, y, epochs=10)

Epoch 1/10
55/55 [==============================] - 26s 262ms/step - loss: 0.5919 - accuracy: 0.6863
Epoch 2/10
55/55 [==============================] - 14s 254ms/step - loss: 0.4187 - accuracy: 0.8506
Epoch 3/10
55/55 [==============================] - 14s 256ms/step - loss: 0.3512 - accuracy: 0.8731
Epoch 4/10
55/55 [==============================] - 14s 257ms/step - loss: 0.3156 - accuracy: 0.8887
Epoch 5/10
55/55 [==============================] - 14s 260ms/step - loss: 0.2939 - accuracy: 0.8962
Epoch 6/10
55/55 [==============================] - 14s 260ms/step - loss: 0.2675 - accuracy: 0.9060
Epoch 7/10
55/55 [==============================] - 14s 259ms/step - loss: 0.2721 - accuracy: 0.8950
Epoch 8/10
55/55 [==============================] - 14s 259ms/step - loss: 0.2438 - accuracy: 0.9152
Epoch 9/10
55/55 [==============================] - 14s 260ms/step - loss: 0.2293 - accuracy: 0.9256
Epoch 10/10
55/55 [==============================] - 14s 259ms/step - loss: 0.2223 - accura

# Evaluate

In [19]:
# test

test_class_images = {
    "bacterial_spot": list(train_dir.glob("bacterial_spot/*")), 
    "healthy": list(train_dir.glob("healthy/*")),
}

test_class_labels = {
    "bacterial_spot": 1,
    "healthy": 0,
}

X_test, y_test = [], []

for image_name, images in test_class_images.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img, (224, 224))
        X_test.append(resized_img)
        y_test.append(test_class_labels[image_name])
        
X_test = np.array(X_test)
y_test = np.array(y_test)

model.evaluate(X_test, y_test)

55/55 [==============================] - 3s 42ms/step - loss: 0.2138 - accuracy: 0.9314


[0.2137550264596939, 0.9313725233078003]

In [23]:
model.save("/content/drive/MyDrive/Colab Notebooks/Plant disease detection/pepper_model.h5", model)

In [30]:
loaded_model = tf.keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/Plant disease detection/pepper_model.h5", custom_objects={'KerasLayer': hub.KerasLayer})

In [37]:
_dir = pathlib.Path("/content/drive/MyDrive/Colab Notebooks/Plant disease detection/")

sample_images_list = list(_dir.glob("pepper_test_images/*"))

for count in range(len(sample_images_list)):
  image = PIL.Image.open(str(sample_images_list[count]))
  image_array = np.array(image.resize((224, 224)))
  print(f"image: {str(sample_images_list[count])}",
        f"\nprediction: {np.argmax(loaded_model.predict(np.array([image_array])))}")

1/1 [==============================] - 0s 18ms/step
image: /content/drive/MyDrive/Colab Notebooks/Plant disease detection/pepper_test_images/image (1257).JPG 
prediction: 0
1/1 [==============================] - 0s 18ms/step
image: /content/drive/MyDrive/Colab Notebooks/Plant disease detection/pepper_test_images/image (702).JPG 
prediction: 1
1/1 [==============================] - 0s 17ms/step
image: /content/drive/MyDrive/Colab Notebooks/Plant disease detection/pepper_test_images/image (701).JPG 
prediction: 1
1/1 [==============================] - 0s 18ms/step
image: /content/drive/MyDrive/Colab Notebooks/Plant disease detection/pepper_test_images/image (1273).JPG 
prediction: 0
